In [1]:
from sqlalchemy import create_engine
import pandas as pd

### Create Database

Open pgAdmin and right-click on your postgres server (mine's postgres 10 bc my Mac refused to work with 11 and 12), but for most of you, it should be 12. Then go to "create" and "database."

<img src="./dbInstructionImages/createDB.png" width="60%">

Then name your database "aq_db" and save.

<img src="./dbInstructionImages/saveDB.png" width="40%">

In [2]:
# Create connection string to database. Note that my port number is 5433.
# Once again this is due to my Mac not playing nice. Most likely, your port number is 5432 so make sure to change.
# Additionally, your username and password are most likely postgres:postgres as they're specified here.

rds_connection_string = "postgres:postgres@localhost:5433/aq_db"

In [3]:
# Set up your database engine

engine = create_engine(f'postgresql://{rds_connection_string}')

In [4]:
# Read our csv into a dataframe using pandas

df = pd.read_csv('./Shane/output_data/AirNowAPI_20200411_073607.csv')
df.head()

,DateObserved,HourObserved,LocalTimeZone,ReportingArea,StateCode,Latitude,Longitude,ParameterName,AQI,AQICategoryNumber,AQICategory,cityRank,cityName,cityCSA,cityState,cityPopulation2018Estimate,cityPopulation2010Census,cityLat,cityLong,cityZip
0,1/1/15,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,30,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
1,1/1/15,0,EST,Newark,NJ,40.7267,-74.1442,PM2.5,65,2,Moderate,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
2,1/2/15,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,24,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
3,1/2/15,0,EST,Newark,NJ,40.7267,-74.1442,PM2.5,56,2,Moderate,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
4,1/3/15,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,8,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001


In [5]:
# Use the dataframe's 'to_sql' method to create a new table named 'city_aq', using the engine we created earlier.
# The first two lines ensure that we drop the table if it already exists so we can create our table using
# our dataframe method. (For example, if our csv data changes and we need to update the table.)

with engine.connect() as connection:
    connection.execute("DROP TABLE IF EXISTS city_aq")

df.to_sql('city_aq',con=engine,index=False)

If you return to pgAdmin, you'll see our newly created table in the database. (If you don't, you may need to right-click on 'Tables' and refresh.)

<img src="./dbInstructionImages/table.png" width="40%">

In [6]:
# Here you can see our table was properly created as we're reading it back into pandas

df_fromPG = pd.read_sql('city_aq',con=engine)
df_fromPG.head()

,DateObserved,HourObserved,LocalTimeZone,ReportingArea,StateCode,Latitude,Longitude,ParameterName,AQI,AQICategoryNumber,AQICategory,cityRank,cityName,cityCSA,cityState,cityPopulation2018Estimate,cityPopulation2010Census,cityLat,cityLong,cityZip
0,1/1/15,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,30,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
1,1/1/15,0,EST,Newark,NJ,40.7267,-74.1442,PM2.5,65,2,Moderate,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
2,1/2/15,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,24,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
3,1/2/15,0,EST,Newark,NJ,40.7267,-74.1442,PM2.5,56,2,Moderate,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001
4,1/3/15,0,EST,Newark,NJ,40.7267,-74.1442,OZONE,8,1,Good,1,New York City,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",NY,22679948,22255491,40.7128,-74.006,10001


Additionally, if you open up the query tool in pgAdmin (right click on aq_db in left column and click 'Query Tool'), you can execute an explicit SQL command to show the table data.

<img src='./dbInstructionImages/querytool.png' width='60%'>

#### With this table now set-up, python-flask can connect to the database, read in the data, and filter it according to our front-end inputs.